In [1]:
import astropy as ap
import healpy as hp
import pandas as pd
import numpy as np
import scipy as sc

/Users/heyfaraday/anaconda/envs/new2/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
hdulist1 = ap.io.fits.open('data/LFI_SkyMap_044-BPassCorrected-field-IQU_0256_R2.01_full.fits')
hdulist2 = ap.io.fits.open('data/pixel_coords_map_nested_ns256.fits')

In [3]:
hdulist1[0].header

SIMPLE  =                    T / Written by IDL:  Mon Oct 19 17:09:28 2015      
BITPIX  =                    8 /Dummy primary header created by MWRFITS         
NAXIS   =                    0 /No data is associated with this header          
EXTEND  =                    T /Extensions may (will!) be present               
CHECKSUM= 'J7dPK6bNJ6bNJ6bN'   / HDU checksum created 2015-10-19T15:09:28       

In [4]:
hdulist1[1].header

XTENSION= 'BINTABLE'           /Written by IDL:  Mon Oct 19 17:09:27 2015       
BITPIX  =                    8 /                                                
NAXIS   =                    2 /Binary table                                    
NAXIS1  =                   12 /Number of bytes per row                         
NAXIS2  =               786432 /Number of rows                                  
PCOUNT  =                    0 /Random parameter count                          
GCOUNT  =                    1 /Group count                                     
TFIELDS =                    3 /Number of columns                               
COMMENT                                                                         
COMMENT  *** End of mandatory fields ***                                        
COMMENT                                                                         
EXTNAME = 'BPassCorrected'     /Extension name                                  
EXTVER  =                   

In [5]:
hdulist2[0].header

SIMPLE  =                    T /Dummy Created by MWRFITS v1.11                  
BITPIX  =                    8 /Dummy primary header created by MWRFITS         
NAXIS   =                    0 /No data is associated with this header          
EXTEND  =                    T /Extensions may (will!) be present               

In [6]:
hdulist2[1].header

XTENSION= 'BINTABLE'           /Binary table written by MWRFITS v1.11           
BITPIX  =                    8 /Required value                                  
NAXIS   =                    2 /Required value                                  
NAXIS1  =                   52 /Number of bytes per row                         
NAXIS2  =               786432 /Number of rows                                  
PCOUNT  =                    0 /Normally 0 (no varying arrays)                  
GCOUNT  =                    1 /Required value                                  
TFIELDS =                    7 /Number of columns in table                      
TFORM1  = 'J       '           /                                                
TFORM2  = 'D       '           /                                                
TFORM3  = 'D       '           /                                                
TFORM4  = 'D       '           /                                                
TFORM5  = 'D       '        

In [7]:
print hdulist1[1].columns
print hdulist1[1].data[0]

ColDefs(
    name = 'TEMPERATURE'; format = 'E'; unit = 'K_CMB'
    name = 'Q_POLARIZATION'; format = 'E'; unit = 'K_CMB'
    name = 'U_POLARIZATION'; format = 'E'; unit = 'K_CMB'
)
(0.0031698239, 1.9594201e-05, -1.8488504e-06)


In [8]:
print hdulist2[1].columns
print hdulist2[1].data[0]

ColDefs(
    name = 'PIXNUM'; format = 'J'
    name = 'GLON'; format = 'D'
    name = 'GLAT'; format = 'D'
    name = 'RA_J2000'; format = 'D'
    name = 'DEC_J2000'; format = 'D'
    name = 'ELON'; format = 'D'
    name = 'ELAT'; format = 'D'
)
(0, 45.0, 0.14920792779581404, 288.29303324362053, 10.792924914087711, 291.53773611464624, 32.874568057877504)


In [9]:
hdulist1[1].data[:]

FITS_rec([( 0.00316982,   1.95942011e-05,  -1.84885039e-06),
       ( 0.0031159 ,   2.00215927e-05,   2.13285603e-08),
       ( 0.00275203,   1.82829390e-05,  -3.22038568e-06), ...,
       ( 0.00265198,   2.30908572e-05,  -5.39131361e-06),
       ( 0.00285171,   1.92052012e-05,  -4.28331532e-06),
       ( 0.00277394,   2.41910766e-05,  -2.20624679e-06)], 
      dtype=(numpy.record, [('TEMPERATURE', '>f4'), ('Q_POLARIZATION', '>f4'), ('U_POLARIZATION', '>f4')]))

In [10]:
hdulist2[1].data[:]

FITS_rec([ (     0,   45.        ,  0.14920793,  288.29303324,  10.79292491,  291.53773611,  32.87456806),
       (     1,   45.17578125,  0.29841687,  288.24139154,  11.01784814,  291.517813  ,  33.10453026),
       (     2,   44.82421875,  0.29841687,  288.07553968,  10.70629897,  291.27095024,  32.820466  ),
       ...,
       (786429,  315.17578125, -0.29841687,  218.54316017, -60.70446295,  238.58601966, -42.75486537),
       (786430,  314.82421875, -0.29841687,  217.87750882, -60.83842998,  238.26279591, -43.01467883),
       (786431,  315.        , -0.14920793,  218.09508128, -60.63385649,  238.27449673, -42.78426963)], 
      dtype=(numpy.record, [('PIXNUM', '>i4'), ('GLON', '>f8'), ('GLAT', '>f8'), ('RA_J2000', '>f8'), ('DEC_J2000', '>f8'), ('ELON', '>f8'), ('ELAT', '>f8')]))

In [11]:
field = np.array(hdulist1[1].data.T)
coords = np.array(hdulist2[1].data.T)

In [12]:
x = np.zeros(np.size(coords))
y = np.zeros(np.size(coords))
f = np.zeros(np.size(field))

In [13]:
for i in xrange(0, 786432):
    x[i] = coords[i][1]
    y[i] = coords[i][2]
    f[i] = field[i][0] # T

In [14]:
unigue_x = sc.unique(x)
unigue_y = sc.unique(y)

In [15]:
data_n = np.column_stack((x, y, f))
data = pd.DataFrame(data_n, columns=['x', 'y', 'f'])